<a href="https://colab.research.google.com/github/JasmineVan/Pyspark_KMean/blob/main/51800128_KMean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Trinh Van Thuong**\
51800128\
Massive Data Processing - Locality Sensive Hashes


---




In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark import SparkContext

sc = SparkContext("local", "LSH")

In [5]:
from pyspark.sql import SQLContext
import pandas as pd

sqlc = SQLContext(sc)

#Đọc DataFrame từ tập tin /content/sample_data/mnist_train_small.csv, chỉ lấy 100 dòng đầu tiên

In [6]:
# Please change file dir
path = '/content/sample_data/mnist_train_small.csv'

In [16]:
df = sqlc.read.csv(path, header=True)
df = df.limit(100)
df.show()

+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+-----+-----+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+---+------+------+-----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+------+---+------+---+---+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+----+-----+---+------+---+----+----+----+----+----+----+----+----+----+-

In [17]:
print(type(df))

<class 'pyspark.sql.dataframe.DataFrame'>


In [18]:
rows = df.count()
print(f"DataFrame Rows count : {rows}")

cols = len(df.columns)
print(f"DataFrame Columns count : {cols}")

DataFrame Rows count : 100
DataFrame Columns count : 785


#Đổi tên cột thứ nhất thành **"label"**, gom các cột còn lại thành cột **"features"** chứa một **Vectors.dense** trên mỗi dòng.

In [20]:
#rename the first column
df = df.withColumnRenamed("6", "label")

In [21]:
import pyspark.sql.functions as f
ColumnName = df.schema.names
columns = [] 
for name in ColumnName:
    columns.append(f.col(name))

In [22]:
#Column combine
df2 = df.withColumn("01", f.array(columns)).select("label", "01")
df2 = df2.withColumnRenamed("01", "features")

In [23]:
df2.show()
df2.printSchema()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    5|[5, 0, 0, 0, 0, 0...|
|    7|[7, 0, 0, 0, 0, 0...|
|    9|[9, 0, 0, 0, 0, 0...|
|    5|[5, 0, 0, 0, 0, 0...|
|    2|[2, 0, 0, 0, 0, 0...|
|    0|[0, 0, 0, 0, 0, 0...|
|    6|[6, 0, 0, 0, 0, 0...|
|    5|[5, 0, 0, 0, 0, 0...|
|    5|[5, 0, 0, 0, 0, 0...|
|    2|[2, 0, 0, 0, 0, 0...|
|    7|[7, 0, 0, 0, 0, 0...|
|    7|[7, 0, 0, 0, 0, 0...|
|    6|[6, 0, 0, 0, 0, 0...|
|    8|[8, 0, 0, 0, 0, 0...|
|    9|[9, 0, 0, 0, 0, 0...|
|    3|[3, 0, 0, 0, 0, 0...|
|    5|[5, 0, 0, 0, 0, 0...|
|    7|[7, 0, 0, 0, 0, 0...|
|    6|[6, 0, 0, 0, 0, 0...|
|    9|[9, 0, 0, 0, 0, 0...|
+-----+--------------------+
only showing top 20 rows

root
 |-- label: string (nullable = true)
 |-- features: array (nullable = false)
 |    |-- element: string (containsNull = true)



In [24]:
#Vector dense
from pyspark.ml.linalg import Vectors
df3 = df2.rdd.map(lambda x: (x['label'], Vectors.dense(x['features']))).toDF(['label', 'features'])
df3.show()
df3.printSchema()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    5|[5.0,0.0,0.0,0.0,...|
|    7|[7.0,0.0,0.0,0.0,...|
|    9|[9.0,0.0,0.0,0.0,...|
|    5|[5.0,0.0,0.0,0.0,...|
|    2|[2.0,0.0,0.0,0.0,...|
|    0|[0.0,0.0,0.0,0.0,...|
|    6|[6.0,0.0,0.0,0.0,...|
|    5|[5.0,0.0,0.0,0.0,...|
|    5|[5.0,0.0,0.0,0.0,...|
|    2|[2.0,0.0,0.0,0.0,...|
|    7|[7.0,0.0,0.0,0.0,...|
|    7|[7.0,0.0,0.0,0.0,...|
|    6|[6.0,0.0,0.0,0.0,...|
|    8|[8.0,0.0,0.0,0.0,...|
|    9|[9.0,0.0,0.0,0.0,...|
|    3|[3.0,0.0,0.0,0.0,...|
|    5|[5.0,0.0,0.0,0.0,...|
|    7|[7.0,0.0,0.0,0.0,...|
|    6|[6.0,0.0,0.0,0.0,...|
|    9|[9.0,0.0,0.0,0.0,...|
+-----+--------------------+
only showing top 20 rows

root
 |-- label: string (nullable = true)
 |-- features: vector (nullable = true)



#Thêm cột **"index"** vào DataFrame để đánh chỉ số dòng tăng dần (gợi ý dùng hàm **monotonically_increasing_id()** )

In [26]:
#Add index column
df3 = df3.withColumn("index", f.monotonically_increasing_id())
df3.show()
df3.printSchema()


+-----+--------------------+-----+
|label|            features|index|
+-----+--------------------+-----+
|    5|[5.0,0.0,0.0,0.0,...|    0|
|    7|[7.0,0.0,0.0,0.0,...|    1|
|    9|[9.0,0.0,0.0,0.0,...|    2|
|    5|[5.0,0.0,0.0,0.0,...|    3|
|    2|[2.0,0.0,0.0,0.0,...|    4|
|    0|[0.0,0.0,0.0,0.0,...|    5|
|    6|[6.0,0.0,0.0,0.0,...|    6|
|    5|[5.0,0.0,0.0,0.0,...|    7|
|    5|[5.0,0.0,0.0,0.0,...|    8|
|    2|[2.0,0.0,0.0,0.0,...|    9|
|    7|[7.0,0.0,0.0,0.0,...|   10|
|    7|[7.0,0.0,0.0,0.0,...|   11|
|    6|[6.0,0.0,0.0,0.0,...|   12|
|    8|[8.0,0.0,0.0,0.0,...|   13|
|    9|[9.0,0.0,0.0,0.0,...|   14|
|    3|[3.0,0.0,0.0,0.0,...|   15|
|    5|[5.0,0.0,0.0,0.0,...|   16|
|    7|[7.0,0.0,0.0,0.0,...|   17|
|    6|[6.0,0.0,0.0,0.0,...|   18|
|    9|[9.0,0.0,0.0,0.0,...|   19|
+-----+--------------------+-----+
only showing top 20 rows

root
 |-- label: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- index: long (nullable = false)



Tạo ra một object từ lớp **KMeans** (pyspark.ml.clustering) với **k=10**, **maxIter=10**

In [29]:
from pyspark.ml.clustering import KMeans, KMeansModel

kmeans = KMeans(k=10)
kmeans.setSeed(1)
kmeans.setMaxIter(10)

KMeans_75c28b4b869b

#Tạo ra model bằng cách cho object ở bước trên xử lý DataFrame

In [30]:
model = kmeans.fit(df3)
print(model.getDistanceMeasure())

euclidean


In [32]:
model.setPredictionCol("newPrediction")
print(model.predict(df3.head().features))

3


In [33]:
centers = model.clusterCenters()
print(len(centers))
print(centers)

10
[array([5.7600e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
       0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0

In [35]:
transformed = model.transform(df3).select("features", "newPrediction")
rows = transformed.collect()
print(rows)

[Row(features=DenseVector([5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 28.0, 59.0, 50.0, 0.0, 23.0, 0.0, 0.0, 32.0, 134.0, 180.0, 254.0, 206.0, 8.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.0, 96.0, 216.0, 233.0, 254.0, 248.0, 215.0, 231.0, 215.0, 215.0, 236.0, 2

In [36]:
transformed.show()

+--------------------+-------------+
|            features|newPrediction|
+--------------------+-------------+
|[5.0,0.0,0.0,0.0,...|            3|
|[7.0,0.0,0.0,0.0,...|            1|
|[9.0,0.0,0.0,0.0,...|            0|
|[5.0,0.0,0.0,0.0,...|            3|
|[2.0,0.0,0.0,0.0,...|            8|
|[0.0,0.0,0.0,0.0,...|            3|
|[6.0,0.0,0.0,0.0,...|            8|
|[5.0,0.0,0.0,0.0,...|            3|
|[5.0,0.0,0.0,0.0,...|            3|
|[2.0,0.0,0.0,0.0,...|            7|
|[7.0,0.0,0.0,0.0,...|            1|
|[7.0,0.0,0.0,0.0,...|            9|
|[6.0,0.0,0.0,0.0,...|            7|
|[8.0,0.0,0.0,0.0,...|            0|
|[9.0,0.0,0.0,0.0,...|            0|
|[3.0,0.0,0.0,0.0,...|            2|
|[5.0,0.0,0.0,0.0,...|            9|
|[7.0,0.0,0.0,0.0,...|            0|
|[6.0,0.0,0.0,0.0,...|            0|
|[9.0,0.0,0.0,0.0,...|            0|
+--------------------+-------------+
only showing top 20 rows

